In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("all_searched_entities_26_08_24.tsv", sep='\t')

In [3]:
data

,ENTITYA,TYPEA,IDA,DATABASEA,ENTITYB,TYPEB,IDB,DATABASEB,EFFECT,MECHANISM,...,MODASEQ,MODIFICATIONB,MODBSEQ,PMID,DIRECT,NOTES,ANNOTATOR,SENTENCE,SCORE,SIGNOR_ID
0,ITCH,protein,Q96J02,UNIPROT,EGFR,protein,P00533,UNIPROT,down-regulates quantity by destabilization,polyubiquitination,...,NaN,NaN,NaN,12226085,YES,NaN,miannu,"In summary, we have shown that CBLC and AIP4 c...",0.471,SIGNOR-272604
1,EREG,protein,O14944,UNIPROT,EGFR,protein,P00533,UNIPROT,up-regulates,binding,...,NaN,NaN,NaN,9419975,YES,Epiregulin may be a mediator of localized cell...,gcesareni,Chemical cross-linking experiments showed that...,0.889,SIGNOR-54351
2,EGFR,protein,P00533,UNIPROT,GRB2,protein,P62993,UNIPROT,up-regulates activity,binding,...,NaN,NaN,NaN,16729043,YES,NaN,lperfetto,We determined interaction partners to all cyto...,0.922,SIGNOR-236327
3,OSI-420,chemical,CID:18924996,PUBCHEM,EGFR,protein,P00533,UNIPROT,down-regulates,chemical inhibition,...,NaN,NaN,NaN,Other,YES,Selleck,gcesareni,NaN,0.800,SIGNOR-195248
4,EGFR,protein,P00533,UNIPROT,JAK1/STAT1/STAT3,complex,SIGNOR-C120,SIGNOR,up-regulates activity,phosphorylation,...,NaN,NaN,NaN,15284024,YES,NaN,NaN,Stimulation of EGFR induces Tyr701 phosphoryla...,0.615,SIGNOR-252088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,afatinib,chemical,CHEBI:61390,ChEBI,EGFR,protein,P00533,UNIPROT,down-regulates,chemical inhibition,...,NaN,NaN,NaN,Other,YES,Selleck,gcesareni,NaN,0.800,SIGNOR-189356
264,EGFR,protein,P00533,UNIPROT,ERBB3,protein,P21860,UNIPROT,up-regulates,phosphorylation,...,NaN,NaN,NaN,7929151,YES,NaN,lperfetto,The erbb3 protein which possesses little or no...,0.653,SIGNOR-34748
265,EGFR,protein,P00533,UNIPROT,STAT5B,protein,P51692,UNIPROT,up-regulates,phosphorylation,...,NaN,NaN,NaN,11751923,YES,NaN,llicata,Novel activation of stat5b in response to epid...,0.827,SIGNOR-113397
266,EGFR,protein,P00533,UNIPROT,CCDC50,protein,Q8IVM0,UNIPROT,down-regulates activity,phosphorylation,...,NaN,NaN,NaN,19059208,YES,NaN,miannu,We also detected tyrosine phosphorylation of Y...,0.424,SIGNOR-262850


In [4]:
relationships = []
for a, b, c in zip(data['ENTITYA'], data['EFFECT'], data['ENTITYB']):
    relationships.append((a, b, c))

In [6]:
import networkx as nx
graph = nx.DiGraph();

for relationship in relationships:
    graph.add_edge(relationship[0], relationship[2], label=relationship[1])

In [17]:
pos = nx.fruchterman_reingold_layout(graph, k=0.00001)

In [12]:
!pip install plotly

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/b3/f6/64fe388089588a0c364b03809443cd628fe0261d1364313fb7aec6bb0ebc/plotly-5.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/b6/cb/b86984bed139586d01532a587464b5805f12e397594f19f931c4c2fbfa61/tenacity-9.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 18.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
import plotly.graph_objects as go

edge_traces = []
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace = go.Scatter(
        x = [x0, x1, None],
        y = [y0, y1, None],
        mode = 'lines',
        line = dict(width = 0.5, color = 'gray'),
        hoverinfo = 'none'
    )
    edge_traces.append(edge_trace)
    
node_trace = go.Scatter(
    
    x = [pos[node][0] for node in graph.nodes()],
    y = [pos[node][1] for node in graph.nodes()],
    mode='markers+text',
    marker = dict(size = 10, color='lightblue'),
    text = [node for node in graph.nodes()],
    textposition = 'top center',
    hoverinfo = 'text',
    textfont=dict(size=7)
    
)

edge_label_trace = go.Scatter(
    
    x = [(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in graph.edges()],
    y = [(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in graph.edges()],
    mode= 'text',
    text=[graph[edge[0]][edge[1]]['label'] for edge in graph.edges()],
    textposition='middle center',
    hoverinfo='none',
    textfont=dict(size=7)
    
)

# Create layout
layout = go.Layout(
    title='Knowledge Graph',
    titlefont_size=16,
    title_x=0.5,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis_visible=False,
    yaxis_visible=False
)

# Create Plotly figure
fig = go.Figure(data=edge_traces + [node_trace, edge_label_trace], layout=layout)

# Show the interactive plot
fig.show()